# Graph GP Classification

## Preamble
This notebook provides an example of Gaussian process classification on a graph using Graph Matern kernel and non-conjugate learning techiques described in the paper.

In [1]:
import tensorflow as tf
import numpy as np
import scipy.special
from sklearn.metrics import accuracy_score
import gpflow
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import networkx as nx
from gpflow.utilities import print_summary
import tensorflow_probability as tfp
import seaborn as sns
from tqdm import tqdm

import sys
import os
project_root = os.path.abspath("../..")
sys.path.append(project_root)
import pandas as pd
import pickle

from efficient_graph_gp.graph_kernels import get_normalized_laplacian
from efficient_graph_gp.gpflow_kernels import GraphDiffusionFastGRFKernel, GraphDiffusionPoFMKernel, GraphDiffusionKernel, GraphGeneralPoFMKernel, GraphGeneralFastGRFKernel
from utils import compute_fro
from cora_utils.preprocessing import load_PEMS, load_cora

In [2]:
dtype = tf.float64
gpflow.config.set_default_float(dtype)
gpflow.config.set_default_summary_fmt("notebook")
tf.get_logger().setLevel('ERROR')

## Load and preprocess the Cora dataset
The nasty details of the dataset preparation are hidden away in `preprocessing` module.

In [3]:
np.random.seed(1)
num_eigenpairs = 500
dataset = 'cora'
cls_number = 7
train_num = int(2485 * 0.8)
test_num = int(2485 * 0.2)
M = train_num // 2

In [4]:
G, data_train, data_test = load_cora(num_train=train_num, num_test=test_num)
adjacency_matrix = nx.to_numpy_array(G) 
x_train, y_train = data_train
x_test, y_test = data_test
Z = x_train[np.random.choice(train_num, M, replace=False)].copy()

## Build a GP model
Here we employ techiques of non-conjugate learning described in the section 3.1 of the paper.

Briefly, we approximate a non-Gaussian posterior with a variational distribution induced by _inducing_ points on the graph.
We choose the inducing points to be all training points.

The model is then the usual `SVGP`.

In [5]:
# -------------------------------
# Run 5 independent seeds
# -------------------------------

SEEDS = [0, 1, 2, 3, 4]
MAX_WALK_LENGTH = 5
seed_accuracies = []
NUM_EPOCHES = 1000
WALKS_PER_NODE = 4096


for seed in SEEDS:
    print(f"\n=== Running seed {seed} ===")
    np.random.seed(seed)
    tf.random.set_seed(seed)

    # 1) Sample M inducing points from the 140 training nodes
    indices = np.random.choice(train_num, M, replace=False)
    Z = x_train[indices].copy()  # shape: (M, 1)

    # 2) Build Graph Diffusion kernel
    graph_kernel = GraphGeneralFastGRFKernel(
        adjacency_matrix=adjacency_matrix,
        walks_per_node= WALKS_PER_NODE,
        p_halt=0.1,
        max_walk_length=MAX_WALK_LENGTH
    )

    # 3) Build SVGP model
    likelihood = gpflow.likelihoods.MultiClass(num_classes=cls_number)
    model = gpflow.models.SVGP(
        kernel=graph_kernel,
        likelihood=likelihood,
        inducing_variable=Z,
        num_latent_gps=cls_number,
        whiten=True,
    )

    # 4) Prepare full-batch training dataset (140 nodes) with infinite repeat
    train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)) \
                             .shuffle(train_num, seed=seed) \
                             .batch(train_num) \
                             .repeat()
    train_iter = iter(train_ds)

    # 5) Optimizer
    optimizer = tf.optimizers.Adam(learning_rate=0.003)

    # 6) Training loop with tqdm
    print("Training SVGP (GRF)…")
    for step in tqdm(range(NUM_EPOCHES), desc=f"Seed {seed} Training", unit="step"):
        Xb, Yb = next(train_iter)
        with tf.GradientTape() as tape:
            loss = model.training_loss((Xb, Yb))  # negative ELBO
        grads = tape.gradient(loss, model.trainable_variables)
        # Filter out any None gradients
        grads_and_vars = [(g, v) for g, v in zip(grads, model.trainable_variables) if g is not None]
        optimizer.apply_gradients(grads_and_vars)

    # 7) Print hyperparameter summary for this seed
    print(f"\nModel hyperparameters (seed={seed}):")
    print_summary(model)

    # 8) Evaluate on the test set
    y_pred_mean, _ = model.predict_y(x_test)
    y_pred = np.argmax(y_pred_mean.numpy(), axis=1).ravel()
    acc = accuracy_score(y_test.ravel(), y_pred)
    print(f"Seed {seed} Test Accuracy: {acc*100:.2f}%")
    seed_accuracies.append((seed, acc))

# After loop, summarize all seeds
print("\n=== Summary over all seeds ===")
for seed, acc in seed_accuracies:
    print(f"Seed {seed}: Accuracy = {acc*100:.2f}%")
mean_acc = np.mean([acc for _, acc in seed_accuracies])
std_acc  = np.std([acc for _, acc in seed_accuracies])
print(f"\nMean accuracy over seeds: {mean_acc*100:.2f}% ± {std_acc*100:.2f}%")



=== Running seed 0 ===


Random walks: 100%|██████████| 2485/2485 [10:31<00:00,  3.94it/s]


Training SVGP (GRF)…


Seed 0 Training: 100%|██████████| 1000/1000 [1:21:10<00:00,  4.87s/step]


Model hyperparameters (seed=0):


...

Training SVGP (GRF)…


Seed 4 Training: 100%|██████████| 1000/1000 [46:33<00:00,  2.79s/step]


Model hyperparameters (seed=4):


name,class,transform,prior,trainable,shape,dtype,value
SVGP.kernel.modulator_vector,Parameter,Identity,,True,"(5,)",float64,"[5.84494413e-01, -1.07972000e+00, 4.83942461e-01..."
SVGP.likelihood.invlink.epsilon,Parameter,Sigmoid,Beta,False,(),float64,0.0010000000000000002
SVGP.inducing_variable.Z,Parameter,Identity,,True,"(994, 1)",float64,[[1.341e+03...
SVGP.q_mu,Parameter,Identity,,True,"(994, 7)",float64,"[[-1.07832, -1.37586, 2.30332..."
SVGP.q_sqrt,Parameter,FillTriangular,,True,"(7, 994, 994)",float64,"[[[8.83613026e-01, 0.00000000e+00, 0.00000000e+00..."


Seed 4 Test Accuracy: 86.52%

=== Summary over all seeds ===
Seed 0: Accuracy = 86.32%
Seed 1: Accuracy = 84.91%
Seed 2: Accuracy = 86.32%
Seed 3: Accuracy = 85.92%
Seed 4: Accuracy = 86.52%

Mean accuracy over seeds: 86.00% ± 0.58%
